In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")

import torch
import transformers
import baukit
from tqdm.auto import tqdm
import json
import os
from src import functional
import numpy as np
import logging
from src import models

from src.utils import logging_utils
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

torch.__version__, transformers.__version__, torch.version.cuda

/home/local_arnab/miniconda3/envs/sae/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


('2.4.1', '4.44.2', '12.1')

In [3]:
from datasets import load_dataset

wiki = load_dataset("wikimedia/wikipedia", "20231101.en")

2024-10-15 12:06:58 numexpr.utils INFO     Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-10-15 12:06:58 numexpr.utils INFO     NumExpr defaulting to 8 threads.
2024-10-15 12:06:58 datasets INFO     PyTorch version 2.4.1 available.
2024-10-15 12:06:58 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2024-10-15 12:06:58 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/wikimedia/wikipedia HTTP/11" 200 164229
2024-10-15 12:06:58 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): s3.amazonaws.com:443
2024-10-15 12:06:58 urllib3.connectionpool DEBUG    https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/wikimedia/wikipedia/wikimedia/wikipedia.py HTTP/11" 404 0
2024-10-15 12:06:58 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/wikimedia/wikipedia HTTP/11" 200 164229
2024-10-15 12:06:58 urllib3.connect

In [4]:
# print(len(wiki["train"]))
# wiki["train"][:5]

In [5]:
# tiny = load_dataset("roneneldan/TinyStories")

In [6]:
# print(len(tiny["train"]))
# tiny["train"][5055]

In [7]:
from src.models import ModelandTokenizer
from nnsight import LanguageModel

# model_name = "openai-community/gpt2-xl"
# model_name = "openai-community/gpt2"
# model_name = "EleutherAI/pythia-410m"
model_name = "google/gemma-2-2b"
# model_name = "meta-llama/Llama-3.2-1B"

mt = ModelandTokenizer(
    model_key=model_name,
    # torch_dtype=torch.float16,
    torch_dtype=torch.float32,
)

# lm = LanguageModel(
#     model_key=model_name,
#     # torch_dtype=torch.float16,
#     device_map = "auto",
#     dispatch=True
# )
# lm.device

2024-10-15 12:07:02 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.55it/s]


2024-10-15 12:07:04 src.models INFO     loaded model </home/local_arnab/Codes/00_MODEL/google/gemma-2-2b> | size: 9972.936 MB | dtype: torch.float32 | device: cuda:0


In [8]:
# # layer_name_format = 'gpt_neox.layers.{}'
# layer_name_format = mt.layer_name_format
# n_embd = mt.n_layer

In [9]:
# mt.embedder_name

In [10]:
# from src.functional import get_module_nnsight
# from src.functional import prepare_input

# prompts = [
#     "A quick brown fox jumps over the lazy",
#     "The capital of France is",
# ]    
# tokens = prepare_input(prompts, tokenizer=mt.tokenizer)

# tokens

In [11]:
# mt.tokenizer.padding_side

In [12]:
# test_tokens = prepare_input(
#     prompts, tokenizer=mt.tokenizer,
#     # truncation=True, max_length=5,
#     padding_side="right",
# )

# test_tokens

In [13]:
# test_tokens = prepare_input(
#     prompts, tokenizer=mt.tokenizer,
#     truncation=True, max_length=5,
#     # padding_side="left",
# )

# test_tokens

In [14]:
# with torch.inference_mode():
#     with mt.trace(test_tokens) as tr:
#         resid_out = get_module_nnsight(mt, layer_name_format.format(3)).output[0].save()
#         # input = get_module_nnsight(mt, mt.embedder_name).input.save()
#         # resid_in = get_module_nnsight(mt, layer_name_format.format(3)).input.save()
#         # output = mt.output.save()

# print(resid_out.shape)
# # resid_out

In [15]:
# with torch.inference_mode():
#     with mt.trace(
#         prompts,
#         invoker_args={"truncation": True, "max_length": 5}
#     ) as tr:
#         resid_out_2 = get_module_nnsight(mt, layer_name_format.format(3)).output[0].save()
#         # input = get_module_nnsight(mt, mt.embedder_name).input.save()
#         # resid_in = get_module_nnsight(mt, layer_name_format.format(3)).input.save()
#         # output = mt.output.save()

# print(resid_out_2.shape)
# # resid_out_2

In [16]:
# torch.allclose(resid_out, resid_out_2)

In [17]:
mt.n_embd*8

18432

In [18]:
from dictionary_learning import ActivationBuffer
from dictionary_learning.training import trainSAE
from src.functional import get_module_nnsight

submodule = get_module_nnsight(mt, mt.layer_name_format.format(6))
activation_dim = mt.n_embd
dictionary_dim = 16384

# data_iter = iter(tiny["train"][:2100000]["text"])
# data_iter = iter(tiny["train"][:10000]["text"])
data_iter = iter(wiki["train"][:1000]["text"])


tiny_buffer = ActivationBuffer(
    data_iter,
    mt,
    submodule,
    d_submodule=activation_dim, 
    n_ctxs=32, 
    ctx_len=256,
    
    refresh_batch_size=32,
    device='cuda:0', 
)

tiny_buffer.config

2024-10-15 12:07:04 git.cmd DEBUG    Popen(['git', 'version'], cwd=/home/local_arnab/Codes/Projects/sae/notebooks, stdin=None, shell=False, universal_newlines=False)
2024-10-15 12:07:04 git.cmd DEBUG    Popen(['git', 'version'], cwd=/home/local_arnab/Codes/Projects/sae/notebooks, stdin=None, shell=False, universal_newlines=False)
2024-10-15 12:07:04 wandb.docker.auth DEBUG    Trying paths: ['/home/local_arnab/.docker/config.json', '/home/local_arnab/.dockercfg']
2024-10-15 12:07:04 wandb.docker.auth DEBUG    No config file found
init self.activations.shape=torch.Size([0, 2304])
init self.read.shape=torch.Size([0])


{'d_submodule': 2304,
 'io': 'out',
 'n_ctxs': 32,
 'ctx_len': 256,
 'refresh_batch_size': 32,
 'out_batch_size': 8192,
 'device': 'cuda:0'}

In [19]:
1000 / 32

31.25

In [20]:
steps = None
for step, act in enumerate(tqdm(tiny_buffer, total=steps)):
    print(step, act.shape)
    break

0it [00:00, ?it/s]You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
0it [00:03, ?it/s]

0 torch.Size([8192, 2304])


In [26]:
from src.functional import free_gpu_cache

free_gpu_cache()

In [27]:
! rm -rf trained_saes
! rm -rf wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
from dictionary_learning.trainers.standard import StandardTrainer
from dictionary_learning.dictionary import AutoEncoder
from src.utils.experiment_utils import set_seed

set_seed(1234)

# train the sparse autoencoder (SAE)
ae = trainSAE(
    data = tiny_buffer,
    trainer_configs= [{
        'trainer' : StandardTrainer,
        'dict_class' : AutoEncoder,
        'activation_dim' : activation_dim,
        'dict_size' : dictionary_dim,
        'lr' : 1e-3,
        'l1_penalty' : 1e-1,
        'warmup_steps' : 1000,
        'resample_steps' : None,
        'seed' : None,
        'wandb_name' : 'StandardTrainer',
        'lm_name': mt.name,
        'layer': submodule,
        'submodule_name': "residual",
    }],
    save_dir="trained_saes",
    save_steps=10,

    use_wandb=True,
    wandb_entity="dl-homeworks",
    wandb_project="test_sae",
    log_steps=5
)

2024-10-15 12:07:08 src.utils.experiment_utils INFO     setting all seeds to 1234


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


2024-10-15 12:07:09 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): api.wandb.ai:443
2024-10-15 12:07:09 urllib3.connectionpool DEBUG    https://api.wandb.ai:443 "POST /graphql HTTP/11" 200 1973
2024-10-15 12:07:09 urllib3.connectionpool DEBUG    https://api.wandb.ai:443 "POST /graphql HTTP/11" 200 381


wandb: Currently logged in as: arnab-api (dl-homeworks). Use `wandb login --relogin` to force relogin


2024-10-15 12:07:10 git.cmd DEBUG    Popen(['git', 'cat-file', '--batch-check'], cwd=/home/local_arnab/Codes/Projects/sae, stdin=<valid stream>, shell=False, universal_newlines=False)


17it [01:34,  5.59s/it]


StandardTrainer-0/frac_variance_explained,▁
StandardTrainer-0/l0,▁███
StandardTrainer-0/l2_loss,▁
StandardTrainer-0/loss,▁
StandardTrainer-0/mse_loss,▁
StandardTrainer-0/sparsity_loss,▁
StandardTrainer-0/frac_variance_explained,nan
StandardTrainer-0/l0,16384
StandardTrainer-0/l2_loss,nan
StandardTrainer-0/loss,nan
StandardTrainer-0/mse_loss,nan


2024-10-15 12:08:47 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): o151352.ingest.sentry.io:443
2024-10-15 12:08:48 urllib3.connectionpool DEBUG    https://o151352.ingest.sentry.io:443 "POST /api/4504800232407040/envelope/ HTTP/11" 200 0


In [24]:
from dictionary_learning.dictionary import AutoEncoder
from src.utils import env_utils

ae = AutoEncoder.from_pretrained(
    path = os.path.join(
        # env_utils.DEFAULT_RESULTS_DIR,
        "trained_saes",
        # mt.name.split("/")[-1],
        # "TinyStories",
        "trainer_0/checkpoints/ae_0.pt"
    ),
    device='cuda:0'
)

/home/local_arnab/Codes/Projects/sae/notebooks/../dictionary_learning/dictionary.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = t.load(path)


In [25]:
ae.encoder

Linear(in_features=2304, out_features=16384, bias=True)